In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin


import statsmodels.api as sm
from scipy import stats
import statsmodels.tools as tools

import warnings
warnings.filterwarnings('ignore')

pd.set_option('max_columns', None)
pd.set_option('max_rows', 100)

# calibration slope
def logit(p):
  return np.log(p/(1-p))

def calibration_slope(ground_truth, probabilities):
  probabilities = np.array(probabilities)
  logit_probabilities = logit(probabilities).reshape(-1,1)
  lr = LogisticRegression(penalty='none', fit_intercept=True).fit(logit_probabilities, ground_truth)
  return lr.coef_.item()

# threshold calculation
class LogisticRegressionWithThreshold(LogisticRegression):
    def predict(self, X, threshold=None):
        if threshold == None: # If no threshold passed in, simply call the base class predict, effectively threshold=0.5
            return LogisticRegression.predict(self, X)
        else:
            y_scores = LogisticRegression.predict_proba(self, X)[:, 1]
            y_pred_with_threshold = (y_scores >= threshold).astype(int)

            return y_pred_with_threshold
    
    def threshold_from_optimal_tpr_minus_fpr(self, X, y):
        y_scores = LogisticRegression.predict_proba(self, X)[:, 1]
        fpr, tpr, thresholds = roc_curve(y, y_scores) 

        optimal_idx = np.argmax(tpr - fpr)

        return thresholds[optimal_idx], tpr[optimal_idx] - fpr[optimal_idx]
    
# roc curve
def PlotROC(y_test,  y_pred_proba, AUCvalue):
    fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
    plt.text(0.3,0, "".join(['AUC =', AUCvalue]), fontsize = 15)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()
    

# frequency plot
def PlotFreq(y_pred_proba):
    y_pred_plot = pd.Series(y_pred_proba)
    y_pred_plot.plot.hist(grid=True, bins=20, rwidth=0.9,
                       color='#607c8e')
    plt.xlabel('Prob')
    plt.ylabel('Frequency')
    plt.grid(axis='y', alpha=0.75)
    plt.show()

In [ ]:
class LogitFit:
    ## input
    def __init__(self, tr_dat, te_dat, nSESInclude):   ## con_var & cate_var are lists of variable names
        self.tr_dat = tr_dat
        self.te_dat = te_dat
        self.nSESInclude = nSESInclude
        
    ### separate x, y
        self.X_tr = self.tr_dat.loc[:,self.tr_dat.columns!='y']
        self.X_te = self.te_dat.loc[:,self.te_dat.columns!='y']
        self.y_tr = self.tr_dat[['y']]
        self.y_te = self.te_dat[['y']]
  
    ### subgrouping
        self.X_te_low = self.X_te[self.X_te["nSES"] == 0]  
        self.y_te_low = self.y_te[self.X_te["nSES"] == 0]
        self.X_te_high = self.X_te[self.X_te["nSES"] == 1]
        self.y_te_high = self.y_te[self.X_te["nSES"] == 1]
        
        self.x_te_f = self.X_te[self.X_te['gender'] == 0]
        self.y_te_f = self.y_te[self.X_te["gender"] == 0]
        self.x_te_m = self.X_te[self.X_te['gender'] == 1]
        self.y_te_m = self.y_te[self.X_te["gender"] == 1]
        
        # subgroups by both gender and nses
        self.x_te_f_low = self.X_te[(self.X_te['gender'] == 0) & (self.X_te["nSES"] == 0)]
        self.x_te_f_high = self.X_te[(self.X_te["gender"] == 0) & self.X_te["nSES"] == 1]
        self.x_te_m_low = self.X_te[(self.X_te['gender'] == 1) & (self.X_te["nSES"] == 0)]
        self.x_te_m_high = self.X_te[(self.X_te['gender'] == 1) & (self.X_te["nSES"] == 1)]
        
        self.y_te_f_low = self.y_te[(self.X_te['gender'] == 0) & (self.X_te["nSES"] == 0)]
        self.y_te_f_high = self.y_te[(self.X_te["gender"] == 0) & self.X_te["nSES"] == 1]
        self.y_te_m_low = self.y_te[(self.X_te['gender'] == 1) & (self.X_te["nSES"] == 0)]
        self.y_te_m_high = self.y_te[(self.X_te['gender'] == 1) & (self.X_te["nSES"] == 1)]
        
        print("Subgrouping")
   
    # whether or not include nSES as predictor
        if nSESInclude == 0:
            self.X_tr = self.X_tr.loc[:,self.X_tr.columns!='nSES']
            self.X_te = self.X_te.loc[:,self.X_te.columns!='nSES']
            self.X_te_low = self.X_te_low.loc[:,self.X_te_low.columns!='nSES']
            self.X_te_high = self.X_te_high.loc[:,self.X_te_high.columns!='nSES']
            self.x_te_f = self.x_te_f.loc[:,self.x_te_f.columns!='nSES']
            self.x_te_m = self.x_te_m.loc[:,self.x_te_m.columns!='nSES']
            
            self.x_te_f_low = self.x_te_f_low.loc[:, self.x_te_f_low.columns != 'nSES']
            self.x_te_f_high = self.x_te_f_high.loc[:, self.x_te_f_high.columns != 'nSES']
            self.x_te_m_low = self.x_te_m_low.loc[:, self.x_te_m_low.columns != 'nSES']
            self.x_te_m_high = self.x_te_m_high.loc[:, self.x_te_m_high.columns != 'nSES']
    
    # model fitting and metrics
        print("\n>>>Model Results")
        self.fit()
        print("\n>>>Model performance")
        self.metrics()
    
    def fit(self):    
        # add constant to training data
        X_tr_const = tools.add_constant(self.X_tr)
        # fit model
        LogitModel = sm.Logit(self.y_tr, X_tr_const).fit()       
        print(LogitModel.summary()) 
        
        # threshold calculation
        self.lrt = LogisticRegressionWithThreshold()
        self.lrt.fit(self.X_tr, self.y_tr)
        self.threshold, optimal_tpr_minus_fpr = self.lrt.threshold_from_optimal_tpr_minus_fpr(self.X_tr, self.y_tr)
        print("threshold=",self.threshold)
        
        self.model = LogisticRegression(solver='liblinear', random_state=0)
        self.model.fit(self.X_tr, self.y_tr)
        
        return LogitModel
    
    def metrics(self):
        
        # pred
        ## prob
        self.y_pred_proba = self.model.predict_proba(self.X_te)[::,1]
        self.y_pred_proba_low = self.model.predict_proba(self.X_te_low)[::,1]
        self.y_pred_proba_high = self.model.predict_proba(self.X_te_high)[::,1]
        self.y_pred_proba_f = self.model.predict_proba(self.x_te_f)[::,1]
        self.y_pred_proba_m = self.model.predict_proba(self.x_te_m)[::,1]
        self.y_pred_proba_f_low = self.model.predict_proba(self.x_te_f_low)[::,1]
        self.y_pred_proba_f_high = self.model.predict_proba(self.x_te_f_high)[::,1]
        self.y_pred_proba_m_low = self.model.predict_proba(self.x_te_m_low)[::,1]
        self.y_pred_proba_m_high = self.model.predict_proba(self.x_te_m_high)[::,1]
        
        ## label using the threshold
        self.pred_threshold = self.lrt.predict(self.X_te, self.threshold)
        self.pred_threshold_low = self.lrt.predict(self.X_te_low, self.threshold)
        self.pred_threshold_high = self.lrt.predict(self.X_te_high, self.threshold)
        self.pred_threshold_f = self.lrt.predict(self.x_te_f, self.threshold)
        self.pred_threshold_m = self.lrt.predict(self.x_te_m, self.threshold)
        self.pred_threshold_f_low = self.lrt.predict(self.x_te_f_low, self.threshold)
        self.pred_threshold_f_high = self.lrt.predict(self.x_te_f_high, self.threshold)
        self.pred_threshold_m_low = self.lrt.predict(self.x_te_m_low, self.threshold)
        self.pred_threshold_m_high = self.lrt.predict(self.x_te_m_high, self.threshold)
        
        # AUC
        print("\n>>>>AUC")
        auc = metrics.roc_auc_score(self.y_te, self.y_pred_proba)
        print("AUC=", auc)
        auc_low = metrics.roc_auc_score(self.y_te_low, self.y_pred_proba_low)
        auc_high = metrics.roc_auc_score(self.y_te_high, self.y_pred_proba_high)
        print("auc_low = ", auc_low,
              "\nauc_high = ", auc_high,
            "\nDisparity in AUC =",abs(auc_low - auc_high))
        auc_f_low = metrics.roc_auc_score(self.y_te_f_low, self.y_pred_proba_f_low)
        auc_f_high = metrics.roc_auc_score(self.y_te_f_high, self.y_pred_proba_f_high)
        auc_m_low = metrics.roc_auc_score(self.y_te_m_low, self.y_pred_proba_m_low)
        auc_m_high = metrics.roc_auc_score(self.y_te_m_high, self.y_pred_proba_m_high)
        print("AUC in f_low group = ",auc_f_low,
             "\n AUC in f_high group = ",auc_f_high,
             "\n AUC in m_low group = ",auc_m_low,
             "\n AUC in m_high group = ",auc_m_high)
        
        # calibration slope
        print("\n>>>>Calibration Slope")
        print("calibration slope =",calibration_slope(self.y_te, self.y_pred_proba))
        cs_low = calibration_slope(self.y_te_low, self.y_pred_proba_low) 
        cs_high = calibration_slope(self.y_te_high, self.y_pred_proba_high)
        dis_cs = abs(cs_low - cs_high)
        print("cs_low = ",cs_low, 
              "\ncs_high = ", cs_high,
            "\nDisparity in calibration slope =",dis_cs)
        print("CS in f_low group = ",calibration_slope(self.y_te_f_low, self.y_pred_proba_f_low),
             "\n CS in f_high group = ",calibration_slope(self.y_te_f_high, self.y_pred_proba_f_high),
             "\n CS in m_low group = ",calibration_slope(self.y_te_m_low, self.y_pred_proba_m_low),
             "\n CS in m_high group = ",calibration_slope(self.y_te_m_high, self.y_pred_proba_m_high))      
        
        # TPR&FPR
        print("\n>>>>TPR&FPR")
        def TprFpr(TrueLabel, PredLable, subgroup):
            CM = confusion_matrix(TrueLabel, PredLable)
            tpr = CM[1,1]/(CM[1,1]+CM[1,0])
            fpr = CM[0,1]/(CM[0,1]+CM[0,0])
            print(subgroup,": \n",
                  "".join(['TPR=', str(tpr), ',','FPR=', str(fpr)]))
    
        TprFpr(self.y_te, self.pred_threshold,"whole")
        TprFpr(self.y_te_low, self.pred_threshold_low,"low")
        TprFpr(self.y_te_high, self.pred_threshold_high,"high")
        TprFpr(self.y_te_f_low, self.pred_threshold_f_low,"f_low")
        TprFpr(self.y_te_f_high, self.pred_threshold_f_high,"f_high")
        TprFpr(self.y_te_m_low, self.pred_threshold_m_low, "m_low")
        TprFpr(self.y_te_m_high, self.pred_threshold_m_high,"m_high")
        
        # roc curve
        PlotROC(self.y_te, self.y_pred_proba, str(auc))
        
        # frequency plot
        PlotFreq(self.y_pred_proba)
        
        

In [ ]:
### fit JHS logit ###

In [ ]:
jhs_tr = pd.read_csv('data/jhs_tr_stratified.csv')
jhs_te = pd.read_csv('data/jhs_te_stratified.csv')

In [ ]:
# M1: without nSES

jhs_tr_m1 = jhs_tr.loc[:,~jhs_tr.columns.isin(['subjid','nbSESpc2score', 'y1', 'y2', 'y3'])]
jhs_te_m1 = jhs_te.loc[:,~jhs_te.columns.isin(['subjid', 'nbSESpc2score', 'y1', 'y2', 'y3'])]

jhs_tr_m1 = jhs_tr_m1.rename(columns={"y_tot": "y"})
jhs_te_m1 = jhs_te_m1.rename(columns={"y_tot": "y"})

jhs_m1 = LogitFit(jhs_tr_m1, jhs_te_m1, nSESInclude = 0)

In [ ]:
## compare predictions

# nses subgroups
dat_pred_low = {'m1_low_prob': jhs_m1.y_pred_proba_low,'m1_low_label': jhs_m1.pred_threshold_low}
m1_pred_low = pd.DataFrame(dat_pred_low)

dat_pred_high = {'m1_high_prob': jhs_m1.y_pred_proba_high,'m1_high_label': jhs_m1.pred_threshold_high}
m1_pred_high = pd.DataFrame(dat_pred_high)

# gender subgroups
dat_pred_f = {'m1_f_prob': jhs_m1.y_pred_proba_f,'m1_f_label': jhs_m1.pred_threshold_f}
m1_pred_f= pd.DataFrame(dat_pred_f)

dat_pred_m = {'m1_m_prob': jhs_m1.y_pred_proba_m,'m1_m_label': jhs_m1.pred_threshold_m}
m1_pred_m = pd.DataFrame(dat_pred_m)

In [ ]:
# M2: with binary nSES

jhs_m2 = LogitFit(jhs_tr_m1, jhs_te_m1, nSESInclude = 1)

In [ ]:
## compare predictions

# nses subgroups
dat_pred_low = {'m2_low_prob': jhs_m2.y_pred_proba_low,'m2_low_label': jhs_m2.pred_threshold_low}
m2_pred_low = pd.DataFrame(dat_pred_low)

dat_pred_high = {'m2_high_prob': jhs_m2.y_pred_proba_high,'m2_high_label': jhs_m2.pred_threshold_high}
m2_pred_high = pd.DataFrame(dat_pred_high)

# gender subgroups
dat_pred_f = {'m2_f_prob': jhs_m2.y_pred_proba_f,'m2_f_label': jhs_m2.pred_threshold_f}
m2_pred_f= pd.DataFrame(dat_pred_f)

dat_pred_m = {'m2_m_prob': jhs_m2.y_pred_proba_m,'m2_m_label': jhs_m2.pred_threshold_m}
m2_pred_m = pd.DataFrame(dat_pred_m)

In [ ]:
# M3: nSES score

jhs_tr_m3 = jhs_tr.loc[:,~jhs_tr.columns.isin(['subjid', 'y1', 'y2', 'y3'])]
jhs_te_m3 = jhs_te.loc[:,~jhs_te.columns.isin(['subjid', 'y1', 'y2', 'y3'])]

jhs_tr_m3 = jhs_tr_m3.rename(columns={"y_tot": "y"})
jhs_te_m3 = jhs_te_m3.rename(columns={"y_tot": "y"})

jhs_m3 = LogitFit(jhs_tr_m3, jhs_te_m3, nSESInclude = 0)

In [ ]:
# compare different predictions

In [ ]:
sub_low = pd.concat([m1_pred_low, m2_pred_low], axis = 1)
jhs_m1.y_te_low.reset_index(drop=True, inplace=True)     ## adding the true value
sub_low = pd.concat([sub_low,jhs_m1.y_te_low], axis = 1)
sub_low = sub_low.loc[(sub_low['m1_low_label'] != sub_low['m2_low_label']), :]

sub_high = pd.concat([m1_pred_high, m2_pred_high], axis = 1)
jhs_m1.y_te_high.reset_index(drop=True, inplace=True)
sub_high = pd.concat([sub_high,jhs_m1.y_te_high], axis = 1)
sub_high = sub_high.loc[(sub_high['m1_high_label'] != sub_high['m2_high_label']), :]

sub_f = pd.concat([m1_pred_f, m2_pred_f], axis = 1)
jhs_m1.y_te_f.reset_index(drop=True, inplace=True)
sub_f = pd.concat([sub_f,jhs_m1.y_te_f], axis = 1)
sub_f = sub_f.loc[(sub_f['m1_f_label'] != sub_f['m2_f_label']), :]

sub_m = pd.concat([m1_pred_m, m2_pred_m], axis = 1)
jhs_m1.y_te_m.reset_index(drop=True, inplace=True)
sub_m = pd.concat([sub_m,jhs_m1.y_te_m], axis = 1)
sub_m = sub_m.loc[(sub_m['m1_m_label'] != sub_m['m2_m_label']), :]

In [ ]:
class SubMetrics:
    def __init__(self, dat):   
        self.dat = dat
        self.true_y = self.dat[['y']]
        self.m1_prob = self.dat.iloc[:, 0]
        self.m1_lab = self.dat.iloc[:, 1]
        self.m2_prob = self.dat.iloc[:, 2]
        self.m2_lab = self.dat.iloc[:, 3]
        
        # AUC
        print("\n>>>>AUC")
        auc_m1 = metrics.roc_auc_score(self.true_y, self.m1_prob)
        auc_m2 = metrics.roc_auc_score(self.true_y, self.m2_prob)
        print("AUC for M1 = ",auc_m1,
             "\n AUC for M2 = ",auc_m2)
        
        # calibration slope
        print("\n>>>>Calibration Slope")
        print("CS for M1 = ",calibration_slope(self.true_y, self.m1_prob),
             "\n CS for M2 = ",calibration_slope(self.true_y, self.m2_prob))      
        
        # TPR&FPR
        print("\n>>>>TPR&FPR")
        def TprFpr(TrueLabel, PredLable, subgroup):
            CM = confusion_matrix(TrueLabel, PredLable)
            tpr = CM[1,1]/(CM[1,1]+CM[1,0])
            fpr = CM[0,1]/(CM[0,1]+CM[0,0])
            print(subgroup,": \n",
                  "".join(['TPR=', str(tpr), ',','FPR=', str(fpr)]))
            
        TprFpr(self.true_y, self.m1_lab,"M1")
        TprFpr(self.true_y, self.m2_lab,"M2")

In [ ]:
SubMetrics(sub_m)

In [ ]:
### JHS: fit logit for two gender separately ###

In [ ]:
class LogitFitGender:
    ## input
    def __init__(self, tr_dat, te_dat, nSESInclude):   ## con_var & cate_var are lists of variable names
        self.tr_dat = tr_dat
        self.te_dat = te_dat
        self.nSESInclude = nSESInclude
        
    ### separate x, y
        self.X_tr = self.tr_dat.loc[:,self.tr_dat.columns!='y']
        self.X_te = self.te_dat.loc[:,self.te_dat.columns!='y']
        self.y_tr = self.tr_dat[['y']]
        self.y_te = self.te_dat[['y']]
  
    ### subgrouping
        self.X_te_low = self.X_te[self.X_te["nSES"] == 0]  
        self.y_te_low = self.y_te[self.X_te["nSES"] == 0]
        self.X_te_high = self.X_te[self.X_te["nSES"] == 1]
        self.y_te_high = self.y_te[self.X_te["nSES"] == 1]
        
   
    # whether or not include nSES as predictor
        if nSESInclude == 0:
            self.X_tr = self.X_tr.loc[:,self.X_tr.columns!='nSES']
            self.X_te = self.X_te.loc[:,self.X_te.columns!='nSES']
            self.X_te_low = self.X_te_low.loc[:,self.X_te_low.columns!='nSES']
            self.X_te_high = self.X_te_high.loc[:,self.X_te_high.columns!='nSES']
            
    
    # model fitting and metrics
        print("\n>>>Model Results")
        self.fit()
        print("\n>>>Model performance")
        self.metrics()
    
    def fit(self):    
        # add constant to training data
        X_tr_const = tools.add_constant(self.X_tr)
        # fit model
        LogitModel = sm.Logit(self.y_tr, X_tr_const).fit()       
        print(LogitModel.summary()) 
        
        # threshold calculation
        self.lrt = LogisticRegressionWithThreshold()
        self.lrt.fit(self.X_tr, self.y_tr)
        self.threshold, optimal_tpr_minus_fpr = self.lrt.threshold_from_optimal_tpr_minus_fpr(self.X_tr, self.y_tr)
        print("threshold=",self.threshold)
        
        self.model = LogisticRegression(solver='liblinear', random_state=0)
        self.model.fit(self.X_tr, self.y_tr)
        
        return LogitModel
    
    def metrics(self):
        
        # pred
        ## prob
        self.y_pred_proba = self.model.predict_proba(self.X_te)[::,1]
        self.y_pred_proba_low = self.model.predict_proba(self.X_te_low)[::,1]
        self.y_pred_proba_high = self.model.predict_proba(self.X_te_high)[::,1]
        
        ## label using the threshold
        self.pred_threshold = self.lrt.predict(self.X_te, self.threshold)
        self.pred_threshold_low = self.lrt.predict(self.X_te_low, self.threshold)
        self.pred_threshold_high = self.lrt.predict(self.X_te_high, self.threshold)
        
        # AUC
        print("\n>>>>AUC")
        auc = metrics.roc_auc_score(self.y_te, self.y_pred_proba)
        print("AUC=", auc)
        auc_low = metrics.roc_auc_score(self.y_te_low, self.y_pred_proba_low)
        auc_high = metrics.roc_auc_score(self.y_te_high, self.y_pred_proba_high)
        print("AUC in low nses group = ",auc_low,
             "\n AUC in high nses group = ",auc_high,
              "\n Disparity in AUC =",abs(auc_low - auc_high))
        
        # calibration slope
        print("\n>>>>Calibration Slope")
        print("calibration slope =",calibration_slope(self.y_te, self.y_pred_proba))
        dis_cs = abs(calibration_slope(self.y_te_low, self.y_pred_proba_low) 
                     - calibration_slope(self.y_te_high, self.y_pred_proba_high))
        print("CS in low nses group = ",calibration_slope(self.y_te_low, self.y_pred_proba_low),
             "\n CS in high nses group = ",calibration_slope(self.y_te_high, self.y_pred_proba_high),
              "\n Disparity in calibration slope =",dis_cs)  
        
        # TPR&FPR
        print("\n>>>>TPR&FPR")
        def TprFpr(TrueLabel, PredLable, subgroup):
            CM = confusion_matrix(TrueLabel, PredLable)
            tpr = CM[1,1]/(CM[1,1]+CM[1,0])
            fpr = CM[0,1]/(CM[0,1]+CM[0,0])
            print(subgroup,": \n",
                  "".join(['TPR=', str(tpr), ',','FPR=', str(fpr)]))
    
        TprFpr(self.y_te, self.pred_threshold,"whole")
        TprFpr(self.y_te_low, self.pred_threshold_low,"low nses")
        TprFpr(self.y_te_high, self.pred_threshold_high,"high nses")
        
        # roc curve
        PlotROC(self.y_te, self.y_pred_proba, str(auc))
        
        # frequency plot
        PlotFreq(self.y_pred_proba)
        
        

In [ ]:
jhs_tr = pd.read_csv('data/jhs_m_tr.csv')
jhs_te = pd.read_csv('data/jhs_m_te.csv')

print(jhs_tr[['y_tot']].value_counts())

print(jhs_te[['y_tot']].value_counts())

In [ ]:
# M1: without nSES

jhs_tr_m1 = jhs_tr.loc[:,~jhs_tr.columns.isin(['subjid','nbSESpc2score', 'gender', 'y1', 'y2', 'y3'])]
jhs_te_m1 = jhs_te.loc[:,~jhs_te.columns.isin(['subjid', 'nbSESpc2score', 'gender','y1', 'y2', 'y3'])]

jhs_tr_m1 = jhs_tr_m1.rename(columns={"y_tot": "y"})
jhs_te_m1 = jhs_te_m1.rename(columns={"y_tot": "y"})

jhs_m1 = LogitFitGender(jhs_tr_m1, jhs_te_m1, nSESInclude = 0)

In [ ]:
# M2: with binary nSES

jhs_m2 = LogitFitGender(jhs_tr_m1, jhs_te_m1, nSESInclude = 1)

In [ ]:
# M3: nSES score

jhs_tr_m3 = jhs_tr.loc[:,~jhs_tr.columns.isin(['subjid', 'y1','gender', 'y2', 'y3'])]
jhs_te_m3 = jhs_te.loc[:,~jhs_te.columns.isin(['subjid', 'y1','gender', 'y2', 'y3'])]

jhs_tr_m3 = jhs_tr_m3.rename(columns={"y_tot": "y"})
jhs_te_m3 = jhs_te_m3.rename(columns={"y_tot": "y"})

jhs_m3 = LogitFitGender(jhs_tr_m3, jhs_te_m3, nSESInclude = 0)